In [ ]:
%pip install --upgrade jax jaxlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled jax-0.4.33


In [ ]:
#https://machinelearningmastery.com/training-stable-diffusion-with-dreambooth/?ref=dailydev
!wget -q https://github.com/kushankjoshi/Tuned-Stable-Diffusion/train_dreambooth.py
!wget -q https://github.com/kushankjoshi/Tuned-Stable-Diffusion/convert_diffusers_to_original_stable_diffusion.py



In [ ]:
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes natsort safetensors xformers
#%pip install --upgrade jax jaxlib
%pip install -q diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 88.6 MB/s eta 0:00:00


In [ ]:
import json
import os

from google.colab import files
import shutil

from natsort import natsorted
from glob import glob

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

In [ ]:
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = #Hugging Face Key
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

In [ ]:
#MODEL_NAME = "runwayml/stable-diffusion-v1-5"
MODEL_NAME="stabilityai/stable-diffusion-xl-base-1.0"
OUTPUT_DIR = "/content/stable_diffusion_weights/shk2"

# Create output directory
!mkdir -p $OUTPUT_DIR

In [ ]:
# The concepts_list is a list of concepts/subject, each represented as a dictionary
concepts_list = [
    {
        "instance_prompt":   "photo of shk2 women",
        "class_prompt":      "photo of pregnant women in her 30s",
        "instance_data_dir": "/content/data/shk",
        "class_data_dir":    "/content/data/men"
    },
]

# Create a directory for each concept according to its instance_data_dir
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

#Dump the concepts_list to a JSON file
with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

In [ ]:
!python3 train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-ema" \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=800 \
  --save_interval=10000 \
  --save_sample_prompt="photo of shk women" \
  --concepts_list="concepts_list.json"

2024-10-12 07:48:02.496005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 07:48:02.516906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 07:48:02.523141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 07:48:03.832172: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(*

In [ ]:
WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(WEIGHTS_DIR)

/content/stable_diffusion_weights/shk/800


In [ ]:
WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(WEIGHTS_DIR)
ckpt_path = WEIGHTS_DIR + "/model.ckpt"

half_arg = ""
fp16 = True
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
ckpt_path = WEIGHTS_DIR + "/model.ckpt"

half_arg = ""
fp16 = True
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path WEIGHTS_DIR  --checkpoint_path ckpt_path half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")


/content/stable_diffusion_weights/shk/800
[*] Converted ckpt saved at /content/stable_diffusion_weights/shk/800/model.ckpt


Save tuned model in google drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Copy the .ckpt file to Google Drive
import shutil
#ckpt_path='/content/stable_diffusion_weights/shk/800/'
drive_path = "/content/drive/MyDrive/StableDiffusion_Models/"
#os.makedirs(drive_path, exist_ok=True)
shutil.copytree(ckpt_path, drive_path, dirs_exist_ok=True)
print(f"[*] Model saved to Google Drive at {drive_path + os.path.basename(ckpt_path)}")

[*] Model saved to Google Drive at /content/drive/MyDrive/StableDiffusion_Models/


Validating tuned model

In [ ]:
model_path = WEIGHTS_DIR

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None,
                                               torch_dtype=torch.float16
                                              ).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()

g_cuda = torch.Generator(device='cuda')
seed = 52362
g_cuda.manual_seed(seed)

In [ ]:
prompt = "photo of shk men in a space suite looking excited after landing in mars "
negative_prompt = ""
num_samples = 4
guidance_scale = 7.5
num_inference_steps = 24
height = 512
width = 512

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)

# Free runtime memory
exit()